In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

## Load Data

In [2]:
pgd = ph.phyloData()

the data of phylo


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [3]:
pgd.search_name("primates")

,name,id
0,Primates,15963


In [4]:
#pgd.fetch_tol_data(15963, limit=25000)

In [5]:
#pgd.add_time()
#pgd.df.head()

### 2) load from raw file

In [6]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [7]:
#pgd.write_prep_data("data/mammal_df.csv", "data/mammal_links.txt")

### 3) load from prepared files

In [8]:
#pgd.load_prep_data("data/Mammalia-15040-df.csv")
pgd.load_prep_data("data/Chordata-2499-df.csv")


### return data for plotting

In [9]:
df = pgd.return_data()
print(df.shape)
df.head()

(10395, 11)


,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End
0,2466,1,0,2499,Chordata,2,0,-0.794184,0.918448,558.0,0.0
1,2499,2,2,14819,Calcichordata,0,0,-0.091174,0.033661,NaN,NaN
2,2499,2,0,14821,Urochordata,8,0,-1.014761,0.366714,555.0,0.0
3,14821,3,0,114489,Octacnemidae,0,0,-0.066112,-0.370996,NaN,NaN
4,14821,3,0,114490,Phlebobranchiata,4,0,-0.344893,-0.201929,NaN,NaN


### hack: get rid of rows with no Begin time or too big Begin time

In [10]:
#df.sort_values(by='Begin', ascending=False).head()

In [11]:
#df = df.loc[df['Begin'].notnull()]
#df = df.loc[df['Begin'] < 500]
#print(df.shape)

## Plot Data

In [12]:
pgp = ph.phyloGraph(df)

the plotting of phylo


In [13]:
pgp.search_name('brata')

                       name     id
38               Vertebrata  14829
7153  Phoebetria palpebrata  89319


In [23]:
pgp.search_name('Homin')

          name     id
641  Hominidae  16299


## BULLSHIT

In [17]:
self = pgp

In [87]:
#focus = 'all'
root_node=15963
focus_node = 16418
root_begin = int(df[df['id']==root_node]['Begin'])
root_begin

55

In [21]:
import pandas as pd

In [25]:
c = 16418
this_row = self.plot_df[self.plot_df['id']==c].squeeze()
this_age = this_row['Begin']
this_age

2.1000000000000001

In [20]:
pick = 15040
mode = 'filter'

In [21]:
# get descendants
kids = []
this_gen = self.links_dict[pick]['children']
while len(this_gen) > 0:
    kids += this_gen
    next_gen = []
    for c in this_gen:
        next_gen += self.links_dict[c]['children']
    this_gen = next_gen
# 
keepers = [pick]
keepers += kids
#
if mode == 'filter':
    self.plot_df = self.df[self.df['id'].isin(keepers)]
    self.plot_df = self.plot_df.reset_index()

In [50]:
import numpy as np

In [32]:
print(self.plot_df.shape)
self.plot_df.head()

(1688, 12)


,index,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End
0,164,14973,10,0,15040,Mammalia,2,0,-3.530605,0.693597,225.00,0.0
1,165,15040,11,2,15989,Triconodonts,0,0,-3.348444,0.438948,190.00,70.0
2,166,15040,11,0,15991,Monotremata,2,0,-2.636896,0.103249,201.30,201.3
3,167,15991,12,0,16250,Tachyglossidae,4,0,-2.788100,-0.445452,23.03,0.0
4,168,16250,13,0,16251,Tachyglossus aculeatus,0,0,-3.588887,-0.799423,NaN,NaN


In [37]:
null_df = self.plot_df[self.plot_df['Begin'].isnull()]

In [92]:
for i, this_row in self.plot_df.iterrows():
#for i, row in self.plot_df.iterrows():
    c = this_row['id']
    #if len(self.links_dict[c]['children']) > 0:
    #if this_row['Begin'] == 55:
    if c == 16293:
        print(this_row)
        print(self.links_dict[c]['children'])
        break

index              624
ancestor         15963
depth               13
extinct              0
id               16293
name        Catarrhini
num_kids             2
phylesis             0
x             -4.56789
y          -0.00107881
Begin               55
End                  0
log_time       4.02535
kin                  0
Name: 4, dtype: object
[16297, 16299, 16300]


In [ ]:
for i, this_row in self.plot_df.iterrows():
    fix_age(i, this_row)

In [ ]:
# Catarhinni
16293

In [93]:
def fix_age(i, this_row):
    c = this_row['id']
    this_age = this_row['Begin']
    parent = self.links_dict[c]['parents']
    parent_df = self.plot_df[self.plot_df['id'].isin(parent)]
    begin_age = float(parent_df['Begin'])
    #
    if (np.isnan(this_age)) | (this_age > (begin_age * 0.95)):
        kids = self.links_dict[c]['children']
        if len(kids) > 0:
            kids_df = self.plot_df[self.plot_df['id'].isin(kids)]
            kids_max = np.nanmax(np.array(kids_df['Begin']))
            if np.isnan(kids_max):
                end_age = float(parent_df['Begin'])
            else:
                end_age = kids_max
        else:
            end_age = kids_max
        #
        self.plot_df.at[i,'Begin'] = np.mean(begin_age, end_age)
        self.plot_df.at[i,'End'] = end_age
        

55.0

0.0

In [28]:
max_depth = 3

In [ ]:
this_gen = self.links_dict[pick]['children']
depth = 0
while len(this_gen) > 0:
    depth += 1
    print(depth)
    next_gen = []
    for c in this_gen:
        

In [27]:
 
        
        
        #
        next_gen += self.links_dict[c]['children']
    if depth >= max_depth:
        break
    else:
        this_gen = next_gen
    

1
2
3
4
5
6


In [ ]:
elif mode == 'focus':
    # add parents
    keepers += self.links_dict[pick]['parents']
    # add kin column
    self.plot_df['kin'] = 0
    self.plot_df['kin'][self.plot_df['id'].isin(keepers)] = 1
    # return list of kin
    return keepers
else:
    print("FAILURE: get_descendants(mode) only valid options are 'filter' and 'focus'")

In [88]:
# take our wrong ones and use raw log
pgp.df = pgp.df.loc[pgp.df['Begin'].notnull()]
pgp.df = pgp.df[pgp.df['Begin'] <= root_begin]
import numpy as np
pgp.df['log_time'] = np.log1p(pgp.df['Begin'])
pgp.df.head()

,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End,log_time
167,15991,12,0,16250,Tachyglossidae,4,0,-2.788100,-0.445452,23.03,0.0,3.179303
170,16250,13,0,117704,Zaglossus bartoni,0,0,-1.814570,-1.129818,23.03,8.0,3.179303
172,15991,12,0,16253,Ornithorhynchus anatinus,0,0,-2.642402,0.753507,9.00,0.0,2.302585
175,15994,12,0,16242,Didelphimorphia,0,0,-5.015041,1.140616,23.03,0.0,3.179303
178,15994,12,0,16245,Dasyuromorphia,0,0,-4.962583,0.572286,23.03,0.0,3.179303


## END BULLSHIT

In [24]:
root_node=15040
focus_node = 16293
#focus_node = 'all'

In [25]:
pgp.create_plot_data(root=root_node, focus=focus_node, Z_dim='log_time', Z_dim_mult=1)
#pgp.create_plot_data(root=root_node, focus=focus_node, add_links=True)

/Users/Seth/Documents/tolGraph/phyloGraph.py:422: RuntimeWarning:

All-NaN slice encountered

/Users/Seth/Documents/tolGraph/phyloGraph.py:477: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting 16293


In [26]:
pgp.plot_df.head()

,index,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End,log_time,kin
0,164,14973,10,0,15040,Mammalia,2,0,-3.530605,0.693597,225.000,0.0,5.420535,0
1,165,15040,11,2,15989,Triconodonts,0,0,-3.348444,0.438948,190.000,70.0,5.252273,0
2,166,15040,11,0,15991,Monotremata,2,0,-2.636896,0.103249,201.300,201.3,5.309752,0
3,167,15991,12,0,16250,Tachyglossidae,4,0,-2.788100,-0.445452,23.030,0.0,3.179303,0
4,168,16250,13,0,16251,Tachyglossus aculeatus,0,0,-3.588887,-0.799423,11.515,0.0,2.526928,0


In [27]:
#pgp.plot_data
pgp.render_plot()
#pgp.render_plot(publish=True, filename="primates2")

In [13]:
#pgp.open_plot()